# Unet天文图像去噪增强

项目主要完成的任务是对天文图像的降噪处理

采用的方法是先在图片上产生噪声，将噪声图片视为训练集，原图像视为标签，通过Unet网络进行训练

训练平台是google colab

### 产生数据

数据集在npy文件夹下。产生原始数据的函数为generate_data.py

通过generate_data.py在npy数据集中寻找所有黑色率小于0.35的204张图片。图片的列表在figurelist.txt中

同时产生一个(204,400,400)的npy文件origin.npy

### 训练

先在本地文件Unet.py中写好训练的代码。

并通过tensorboard产生logs文件，在tensorboard提供的本地网页得到可视化的网络结构。

之后再google colab中上传origin.npy数据集，并在google colab用本笔记中的代码进行训练。

得到traing_Time43.h5网络模板文件

以及训练数据GolbalLoss.txt文件

### 预测

再次使用generate_data.py文件避开之前采用的npy图片产生evallist.txt文件得到一些可用于预测的文件。

通过predict.py 读取evallist.txt得到数据，用和训练类似的方法产生噪声文件，并将预测的结果和噪声图片放入result文件夹中

通过result文件进行数据可视化

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!nvidia-smi

Fri Jul  2 01:26:54 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P0    31W /  70W |   8666MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### U-net网络结构

U型对称结构，左侧是卷积层，右侧是上采样层。原文的Unet结构中，包含4个convolutional layer和对应的4个up sampling layer。

![title](train.png)
在原有Unet的基础上在图片拼接的第一层中加入了exptim的影响。


In [8]:
from numpy.lib.shape_base import expand_dims
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np

IMG_WIDTH = 256
IMG_HEIGHT = 256
IMG_CHANNELS = 1
def lrelu(x):
    return tf.maximum(x*0.2, x)

def Loss(y_true, y_pred):
    return tf.reduce_mean(tf.abs(y_true - y_pred))


def InputBlock(x, filters, time, kernel_size = 3, strides = 1, padding = 'same'):
    conv_1 = layers.Conv2D(filters= filters, kernel_size= kernel_size, \
        strides= strides, padding= padding, activation= lrelu, name= 'g_conv' + time + '_1')(x)
    return layers.Conv2D(filters=filters, kernel_size=kernel_size,\
        strides= strides, padding= padding, activation= lrelu, name= 'g_conv' + time + '_2')(conv_1)

def UpsampleAndConcat(x1, x2, filters, exp_time=None, exp=False):
    output_channels = filters * 2
    deconv = layers.Conv2DTranspose(filters= filters, kernel_size= 2, \
                          strides= 2, padding= 'same')(x1)  # 上采样（转置卷积方式）
    output_channels = filters
    if not exp:
        deconv_output = tf.concat([deconv, x2], 3)
        deconv_output.set_shape([None, None, None, output_channels * 2])
    if exp:
        cons = tf.fill(tf.shape(deconv), exp_time)
        c = tf.cast(tf.slice(cons, [0, 0, 0, 0], [-1, -1, -1, 1]), dtype= tf.float32)
        deconv_output = tf.concat([deconv, x2, c], 3)
        deconv_output.set_shape([None, None, None, output_channels * 2])
    return deconv_output

def ContractingPathBlock(x, filters, time, kernel_size = 3, strides = 1, padding = 'same'):
    down_sampling = layers.MaxPool2D((2, 2))(x)
    conv_1 = layers.Conv2D(filters= filters, kernel_size= kernel_size \
        , strides= strides, padding= padding, activation= lrelu, name= 'g_conv' + time + '1')(down_sampling)
    return layers.Conv2D(filters= filters, kernel_size = kernel_size \
        , strides= strides, padding= padding, activation= lrelu, name= 'g_conv' + time + '2')(conv_1)

def ExpansivePathBlock(x, con_feature, filters, e, time, exp = False, kernel_size = 3 \
    , strides = 1, padding = 'same'):
    concat_feature = UpsampleAndConcat(x, con_feature, filters= filters, exp_time= e, exp= exp)
    conv_1 = layers.Conv2D(filters= filters, kernel_size= kernel_size \
        , strides= strides, padding= padding, activation= lrelu, name= 'g_conv' + time + '1')(concat_feature)
    return layers.Conv2D(filters= filters, kernel_size= kernel_size \
        , strides= strides, padding= padding, activation= lrelu, name= 'g_conv' + time + '2')(conv_1)

def UNet(input_shape, e):
    inputs = layers.Input(input_shape)

    input_block = InputBlock(inputs, filters= 32, time= '1')

    conv_1 = ContractingPathBlock(input_block, filters= 64, time= '2')
    conv_2 = ContractingPathBlock(conv_1, filters= 128, time= '3')
    conv_3 = ContractingPathBlock(conv_2, filters= 256, time= '4')
    conv_4 = ContractingPathBlock(conv_3, filters= 512, time= '5')

    exp_4 = ExpansivePathBlock(conv_4, conv_3, filters= 256, e= e, time= '6')
    exp_3 = ExpansivePathBlock(exp_4, conv_2, filters= 128, e= e, time= '7')
    exp_2 = ExpansivePathBlock(exp_3, conv_1, filters= 64, e= e, time= '8')
    exp_1 = ExpansivePathBlock(exp_2, input_block, filters= 32, e= e, time = '9')

    outputs = layers.Conv2D(filters= 1, kernel_size= 1, activation= None, name = 'g_conv10')(exp_1)
    return tf.keras.Model(inputs= [inputs], outputs= [outputs])

def return_noise(data, exp_time, ratio= 2, dk = 3, ron = 7):
    width, height = data.shape[0:2]
    img = data * exp_time

    DN = np.random.normal(0, np.sqrt(dk*exp_time/((60*60)*ratio)), (width, height))
    RON = np.random.normal(0, ron, (width, height))
    SN = np.random.poisson(np.abs(img/ratio))

    # 以上是噪声基本格式
    noise_img = (SN + RON + DN)/(exp_time/ratio)
    noise_img = np.where(data == 0.00000000e+00, 0.00000000e+00, noise_img)
    # 使用where防止浮点数0.0被加噪

    return noise_img

def black_level(arr, max_num, level=0.3):
    """Prevent to have an image with more than some percentage of zeroes as input
       level - percentage <0,1>; 0.1/10% default"""
    arr = list(np.hstack(arr))
    per = arr.count(0)/len(arr)
    if max_num > 10:
        level = 0.35
    if per < level:
        return True
    else:
        return False

def Expand_Dim(src):
    src = expand_dims(src, axis= 0)
    src = expand_dims(src, axis= -1)
    return src

def generate_data(src, IMG_WIDTH = 256, IMG_HEIGHT = 256):
    H, W = src.shape[0:2]
    zero_level = False
    max_num = 0
    while not zero_level:
        xx = np.random.randint(0, H - IMG_HEIGHT)
        yy = np.random.randint(0, W - IMG_WIDTH)
        arr = src[xx:xx + IMG_HEIGHT, yy:yy + IMG_WIDTH]
        zero_level = black_level(arr, max_num)
        max_num += 1
    out = src[xx:xx + IMG_HEIGHT, yy:yy + IMG_WIDTH]
    out = np.where(out < 0.00000000e+00, 0.00000000e+00  , out)
    return out

### 一些天文图像处理的工具函数

generate_data 用于产生数据，每次在图片中随机寻找一个256*256的数据，通过blacklevel函数判断图像黑色占原图的比例，设置一个阈值之后得到一张亮度合适的图片，作为输入数据。

return_noise用于产生噪声

In [ ]:
home = '/content/gdrive/MyDrive'
origin_data_path= '/Origin.npy' 
Origins = np.load(home + origin_data_path)
Origins.shape

(204, 400, 400)

### 网络的搭建

用tensorflow搭建网络，summary函数中可以再次得到网络结构。

优化器采用adam，学习率为1e-4

In [9]:
network_path = None
if network_path is None: # the first time
    model = UNet(input_shape=(IMG_WIDTH, IMG_HEIGHT, IMG_CHANNELS), e= 674)
    model.summary()
else:
  model = tf.keras.models.load_model(home + network_path, custom_objects = {'lrelu' : lrelu, 'Loss' : Loss})
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss = Loss, metrics=['accuracy'])

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 1) 0                                            
__________________________________________________________________________________________________
g_conv1_1 (Conv2D)              (None, 256, 256, 32) 320         input_1[0][0]                    
__________________________________________________________________________________________________
g_conv1_2 (Conv2D)              (None, 256, 256, 32) 9248        g_conv1_1[0][0]                  
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 128, 128, 32) 0           g_conv1_2[0][0]                  
______________________________________________________________________________________________

In [ ]:
def text_write_data(file, l):
    """save parameters into text file"""
    file = open(home + file, "a+")
    for name in l:
        file.write(str(name) + "\n")
    file.close

### 训练方法

从数据集中导入。

每次打乱数据集的顺序。

从数据集中的每张图片中随机产生5张噪声图片进行训练。

每50张图片记录一次Loss

总共训练了43轮

其中在本地的code文件中采用了tesorboard产生了网络的结构可视化文件（也就是网络结构中的那张图。）


In [ ]:
batch_size = 1
epochs = 1
def Expand(x):
  return np.expand_dims(np.expand_dims(x, axis= 0), axis= -1)
Loss = []
g_Loss = []
Lass_time = 20
for Time in range(Lass_time, 5000):
    cnt = 0
    for origin in np.random.permutation(Origins):
        out = generate_data(origin)
        out_train = Expand(out)
        for r in range(2, 6):
            img = return_noise(out, 674, ratio= r)
            tf.keras.backend.set_value(model.optimizer.lr, 1e-4)
            results = model.fit(Expand(img), out_train, batch_size= batch_size, epochs= epochs)
            g_Loss.append(results.history['loss'][-1])
        
        cnt += 1
        if cnt % 50 == 0:
          print(cnt)
          Loss.append(np.mean(g_Loss))
          g_Loss = []
    if Time % 1 == 0:
        network_path = '/traing'+ '_Time' + str(Time) + '_loss' + str(results.history['loss'][-1]) + '_acu' \
            + str(results.history['accuracy'][-1])
        model.save(home + network_path + '.h5')
        text_write_data("/GlobalLoss.txt", Loss)
        Loss = []


流式输出内容被截断，只能显示最后 5000 行内容。
1/1 [==============================] - 0s 45ms/step - loss: 0.3772 - accuracy: 0.0820
150
1/1 [==============================] - 0s 50ms/step - loss: 0.3969 - accuracy: 0.3042
200
1/1 [==============================] - 0s 52ms/step - loss: 0.4689 - accuracy: 0.1770
50
1/1 [==============================] - 0s 46ms/step - loss: 0.3701 - accuracy: 0.0813
100
1/1 [==============================] - 0s 49ms/step - loss: 0.4871 - accuracy: 0.1154
150
1/1 [==============================] - 0s 46ms/step - loss: 0.3687 - accuracy: 0.0782
200
1/1 [==============================] - 0s 48ms/step - loss: 0.3331 - accuracy: 0.2454
50
1/1 [==============================] - 0s 48ms/step - loss: 0.2947 - accuracy: 0.2711
100
1/1 [==============================] - 0s 46ms/step - loss: 0.4522 - accuracy: 0.1272
150
1/1 [==============================] - 0s 44ms/step - loss: 0.2809 - accuracy: 0.2682
200
1/1 [==============================] - 0s 45ms/step - loss: 0.3263 - accura